In [6]:
import pandas as pd

In [7]:
DF=pd.read_csv("Repalce_NA_with0_SDIdata.csv")

In [8]:
DF1=DF.copy()
DF1['net_asset']=DF1['asset']-DF1['liab']
DF1['net_asset']
DF1 = DF1[DF1.columns.difference(['asset', 'liab'])]
DF1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435110 entries, 0 to 435109
Columns: 763 entries, ABCUBK to webaddr
dtypes: float64(760), int64(2), object(1)
memory usage: 2.5+ GB


In [9]:
DF.head()

,Unnamed: 0,cert,P3GTY,P3GTYGNM,P9GTYPAR,P9GTY,P9GTYGNM,NAGTYPAR,NAGTY,NAGTYGNM,...,SZUCCON,SZUCCI,SZUCOTH,UCSZRES,UCSZHEL,UCSZCRCD,UCSZAUTO,UCSZCON,UCSZCI,UCSZOTH
0,0,35260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,4829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,4829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,4829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
x_variables=list(DF1.columns) 
x_variables.remove('net_asset')

In [12]:
import pandas as pd
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 10

In [13]:
from sklearn.linear_model import Ridge
def ridge_regression(data, predictors, alpha):
    #Fit the model
    ridgereg = Ridge(alpha=alpha,normalize=True)
    ridgereg.fit(data[predictors],data['net_asset'])
    y_pred = ridgereg.predict(data[predictors])
    
    #Return the result in pre-defined format
    rss = sum((y_pred-data['net_asset'])**2)
    ret = [rss]
    ret.extend([ridgereg.intercept_])
    ret.extend(ridgereg.coef_)
    return ret

In [14]:
#Initialize predictors to be set of 15 powers of x
predictors=x_variables

#Set the different values of alpha to be tested
alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]

#Initialize the dataframe for storing coefficients.
col = ['rss','intercept'] + x_variables
ind = ['alpha_%.2g'%alpha_ridge[i] for i in range(len(alpha_ridge))]
coef_matrix_ridge = pd.DataFrame(index=ind, columns=col)
for i in tqdm(range(len(alpha_ridge))):
    coef_matrix_ridge.iloc[i,] = ridge_regression(DF1, predictors, alpha_ridge[i])



  0%|          | 0/10 [00:00<?, ?it/s]

ValueError: could not convert string to float: 'www.1880bank.com'

In [ ]:
coef_matrix_ridge

In [ ]:
(coef_matrix_ridge== 0).astype(int).sum(axis=1)

# for alpha=10
bkprem    Bank premises and fixed assets
eqconsub   Equity, minor interest in consolidated subs
intangw   Goodwill
intan     Goodwill and other intangibles
eqsur    Surplus
RBCT1J    Tier one (core) capital
eq	      Bank equity capital
eqtot     Total equity capital
RWAJT    Total risk weighted assets adjusted
RBCT1J    Tier one (core) capital

In [17]:
from sklearn.linear_model import Lasso
def lasso_regression(data, predictors, alpha):
    #Fit the model
    lassoreg = Lasso(alpha=alpha,normalize=True, max_iter=1e5)
    lassoreg.fit(data[predictors],data['net_asset'])
    y_pred = lassoreg.predict(data[predictors])
    
    
    #Return the result in pre-defined format
    rss = sum((y_pred-data['net_asset'])**2)
    ret = [rss]
    ret.extend([lassoreg.intercept_])
    ret.extend(lassoreg.coef_)
    return ret

In [ ]:
#Initialize predictors to all 15 powers of x
predictors=x_variables


#Define the alpha values to test
alpha_lasso = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]

#Initialize the dataframe to store coefficients
col = ['rss','intercept'] + x_variables
ind = ['alpha_%.2g'%alpha_lasso[i] for i in range(len(alpha_lasso))]
coef_matrix_lasso = pd.DataFrame(index=ind, columns=col)

#Iterate over the 10 alpha values:
for i in range(len(alpha_lasso)):
    coef_matrix_lasso.iloc[i,] = lasso_regression(DF1, predictors, alpha_lasso[i])

In [ ]:
coef_matrix_lasso

In [ ]:
coef_matrix_lasso_nonzero = coef_matrix_lasso.loc[:, (coef_matrix_lasso != 0).any(axis=0)]

coef_matrix_lasso_nonzero = coef_matrix_lasso_nonzero.sort_values(by ='alpha_10', axis=1,ascending=False)
coef_matrix_lasso_nonzero

# alpha =10
eq	      Bank equity capital
eqconsub   Equity, minor interest in consolidated subs
eqtot     Total equity capital
RBCT1J    Tier one (core) capital
bkprem    Bank premises and fixed assets
intangw   Goodwill
intan     Goodwill and other intangibles
eqsur    Surplus
RWAJT    Total risk weighted assets adjusted